In [58]:
import pandas as pd
import numpy as np
####

Country = "Hungary"
Country_abbr = "Hun" # Pol, Hun
text_name = "4877855988"+".txt"

####


df = pd.read_csv("../data/" + Country + '_DataFrame.csv')
''' Select data frame that ends at 2020-03 '''
end_index = None
for index in range(len(df)):
    if df.iloc[index, 0] == "2020-03":
        end_index = index
        break
assert (end_index is not None)


''' Scale the data '''
def scale_series(series, mean, std):
    return (series - mean)/std
for col_num in range(1, len(df.columns)):
    df.iloc[:, col_num] = scale_series(df.iloc[:, col_num], df.iloc[end_index: , col_num ].mean(), df.iloc[end_index: , col_num ].std())


In [59]:

file_path = "./Output/" + Country_abbr +"/corr/"


dict_weight = dict()

with open(file_path+text_name, "r") as f:
    is_Weight = False
    line = f.readline()
    while len(line.strip()):
        if line.strip().startswith("_") or line.strip().startswith("-"):
            line = f.readline()
            continue
        if is_Weight:
            
            name, weight = line.strip().split(":")
            name = name.strip()
            weight = weight.strip()
            weight = float(weight)
            dict_weight[name] = weight
        else:
            if line.startswith("Weight"):
                is_Weight = True
        line = f.readline()


In [61]:

df = df.dropna().reset_index()

FCI = np.matmul(df[dict_weight.keys()],np.square(list(dict_weight.values())))
Date = df["Date"]
df_output = pd.DataFrame({"Date":Date, "FCI":FCI})
df_output.to_csv("Output/" + Country_abbr+"/FCI.csv", index = False)


In [ ]:
df_output


,Date,FCI
0,2014-04,0.098120
1,2014-05,0.552636
2,2014-06,0.255254
3,2014-07,0.036479
4,2014-08,0.271209
...,...,...
104,2022-12,-1.153403
105,2023-01,-0.701535
106,2023-02,-0.662976
107,2023-03,-0.711341
